In [1]:
# import pandas as pd
# anno_1 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\hanhui_oc.xlsx'
# anno_2 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\lsy_oc.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','overall_consistency'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','overall_consistency'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [16]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)   
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr,rank1,rank2

In [11]:
import json
jsonpath = 'C:\\Users\\hh\\Desktop\\HighlyHumanLikeVEB\\Human_anno\\overall_consistency_annotation.json'
with open(jsonpath,'r') as f:
    oc = json.load(f)

In [12]:
import json


import pandas as pd
anno_1 = 'C:\\Users\\hh\\Desktop\\HighlyHumanLikeVEB\\Human_anno\\humananno_res\\hanhui_oc_2.xlsx'

anno_1oc =pd.read_excel(anno_1, usecols=['Model','new_overall'])
anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)



indexlst = []
for i in range(0,len(oc)):
    startindex = 5*i
    endindex = 5*(i+1)
    for idnex in range(startindex,endindex):
        model = anno_1oc['Model'][idnex].lower()
        if anno_1oc['new_overall'][idnex]!=None:
            indexlst.append(i)
            break

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(len(oc)):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/len(oc)
# gptvsanno2_spearman = 3*gptvsanno2_spearman/len(oc)
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/len(oc)
# gptvsannomean = 3*gptvsannomean/len(oc)
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [25]:
gptvsanno1_spearman = np.zeros(len(oc))
gptvsanno2_spearman = np.zeros(len(oc))
anno1vsanno2_spearman = np.zeros(len(oc))
gptvsannomean_spearman = np.zeros(len(oc))
badeval = []

for i in indexlst:
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    anno1 = human_anno[:,0]
    anno2 = human_anno[:,1]
    annomean = (anno1 + anno2)/2
    

    gptvsanno1,a,b = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    print(gpt4o_eval_rs,a)
    print(anno1,b)
    print(i,gptvsanno1)
    print("-----------------------------------------------------")
    gptvsanno2,c,d = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2,e,f = calculate_spearman_manual(anno1,anno2)
    gptvsannomean,g,h = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    
    gptvsanno1_spearman[i] = gptvsanno1
    gptvsanno2_spearman[i] = gptvsanno2
    anno1vsanno2_spearman[i] = anno1vsanno2
    gptvsannomean_spearman[i] = gptvsannomean

    if gptvsannomean < 0.5:
        badeval.append(i)



print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.sum()/len(indexlst))
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.sum()/len(indexlst))
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.sum()/len(indexlst))
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.sum()/len(indexlst))
print("Bad Evaluation: ",badeval)

[2 4 4 3 4] [5.0, 2.0, 2.0, 4.0, 2.0]
[3 5 4 4 5] [5.0, 1.5, 3.5, 3.5, 1.5]
0 0.85
-----------------------------------------------------
[2 5 2 4 2] [4.0, 1.0, 4.0, 2.0, 4.0]
[2 5 3 4 5] [5.0, 1.5, 4.0, 3.0, 1.5]
1 0.575
-----------------------------------------------------
[2 3 2 4 4] [4.5, 3.0, 4.5, 1.5, 1.5]
[5 5 2 4 5] [2.0, 2.0, 5.0, 4.0, 2.0]
2 0.30000000000000004
-----------------------------------------------------
[3 2 4 5 5] [4.0, 5.0, 3.0, 1.5, 1.5]
[3 3 3 5 3] [3.5, 3.5, 3.5, 1.0, 3.5]
3 0.65
-----------------------------------------------------
[2 5 5 5 5] [5.0, 2.5, 2.5, 2.5, 2.5]
[3 5 5 5 4] [5.0, 2.0, 2.0, 2.0, 4.0]
4 0.85
-----------------------------------------------------
[1 5 5 5 5] [5.0, 2.5, 2.5, 2.5, 2.5]
[2 4 5 5 3] [5.0, 3.0, 1.5, 1.5, 4.0]
5 0.775
-----------------------------------------------------
[5 5 1 4 5] [2.0, 2.0, 5.0, 4.0, 2.0]
[5 5 3 4 5] [2.0, 2.0, 5.0, 4.0, 2.0]
6 1.0
-----------------------------------------------------
[5 5 5 5 3] [2.5, 2.5, 2.